In [0]:
%run ../utils/common

In [0]:
dbutils.widgets.text("environment", "", "")
environment = dbutils.widgets.get("environment")

In [0]:
catalog_name = settings[environment]['catalog_name']


In [0]:
spark.sql(f"""
CREATE OR REPLACE TABLE {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_flowthrough_golive 
TBLPROPERTIES (
  'DELTA.AUTOOPTIMIZE.OPTIMIZEWRITE' = 'TRUE',
  'DELTA.AUTOOPTIMIZE.AUTOCOMPACT' = 'TRUE'
)
AS
WITH upload AS (
    SELECT a.business_unit, a.dc_bs_id as dc_id, a.product_id, b.total_shelf_life, a.dc_ft_id 
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_flowthrough_golive a
    LEFT JOIN {catalog_name}.udp_wcm_gold_vcm_dwh.d_product b
        ON a.product_id = b.product_id
),
mapping_dc AS (
    SELECT DISTINCT b.business_unit, a.dc_id, a.dc_ft_id
    FROM {catalog_name}.udp_wcm_gold_db_winare_nonfresh_dc.d_cfg_winare_dc_golive a
    CROSS JOIN (SELECT EXPLODE(array('2000', '6000')) AS business_unit) b
    WHERE 
    a.replenishment_mode = 'BS' AND 
    a.business_unit = '2000' AND a.dc_ft_id IS NOT NULL
),
main AS (
    SELECT 
        a.business_unit, a.dc_id, b.product_id, b.total_shelf_life, a.dc_ft_id
    FROM mapping_dc a
    CROSS JOIN (
        SELECT product_id, total_shelf_life 
        FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_product 
        WHERE total_shelf_life <= 120   
            AND mch2_id IN ('102','201','202','203')
            AND manufacturer_id <> '1012'
    ) b
)
SELECT *
FROM main;
""")

In [0]:
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_flowthrough_golive_his
(
    calday DATE,
    business_unit STRING,
    dc_id STRING,
    product_id STRING,
    total_shelf_life FLOAT,
    dc_ft_id STRING
)
USING DELTA
TBLPROPERTIES (
  'delta.autoOptimize.optimizeWrite' = 'true',
  'delta.autoOptimize.autoCompact' = 'true'
)
""")


In [0]:
spark.sql(f"""
DELETE FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_flowthrough_golive_his 
WHERE calday = DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS);
""")

In [0]:
spark.sql(f"""
INSERT INTO {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_flowthrough_golive_his
SELECT 
    DATE(CURRENT_TIMESTAMP() + INTERVAL 7 HOURS) AS calday,
    business_unit, 
    dc_id, 
    product_id,
    total_shelf_life,
    dc_ft_id
FROM {catalog_name}.udp_wcm_gold_vcm_dwh.d_winare_flowthrough_golive        
""")